In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import requests

In [12]:
#cấu hình chrome driver
chrome_options = Options()
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--headless")
chrome_options.add_argument('--blink-settings=imagesEnabled=false')
chrome_options.add_argument("--disable-gpu")

In [13]:
#khởi tạo driver
driver = webdriver.Chrome(chrome_options=chrome_options, executable_path="chromedriver")

C:\Users\ntu96\AppData\Local\Temp\ipykernel_18296\3663994960.py:2: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chrome_options, executable_path="chromedriver")


In [14]:
#lấy tọa độ từ địa chỉ
def get_location(address):
    key='tUuAAEMo5QSEJdJ39e4nxVLGr904SxhF'
    url='https://www.mapquestapi.com/geocoding/v1/address?key='
    main_url=url+key+'&location='+address
    r=requests.get(main_url)
    data=r.json()['results'][0]
    location=data['locations'][0]
    lat=location['latLng']['lat']
    long=location['latLng']['lng']
    return (lat,long)

In [15]:
# xác định số lượng trang
start_page = 1
end_page = 70

# tạo danh sach chứa các thông tin
links = []
prices = []
areas = []
rooms = []
toilets = []
addresses = []
lats = []
longs = []

# quét tất cả trang
for page in range(start_page, end_page+1):

    url = f'https://mogi.vn/da-nang/mua-nha-dat?cp={page}'
    driver.get(url)

    # Wait for the page to load
    time.sleep(0.5)

    # Get the list of all listings
    listings = driver.find_elements(By.CLASS_NAME,'prop-info')
    #links in listings
    linksInListings =[] 
    # trích xuat thông tin từ các listing
    for listing in listings:
        # lấy thông tin
        link = listing.find_element(By.CLASS_NAME,'link-overlay').get_attribute('href')
        price = listing.find_element(By.CLASS_NAME,'price').text
        area = listing.find_element(By.CLASS_NAME,'prop-attr').find_elements(By.TAG_NAME,'li')[0].text
        room = listing.find_element(By.CLASS_NAME,'prop-attr').find_elements(By.TAG_NAME,'li')[1].text
        toilet = listing.find_element(By.CLASS_NAME,'prop-attr').find_elements(By.TAG_NAME,'li')[2].text
        # thêm thông tin vào danh sách
        links.append(link)
        prices.append(price)
        areas.append(area)
        rooms.append(room)
        toilets.append(toilet)
        linksInListings.append(link)
    # trích xuất địa chỉ từ các link
    for link in linksInListings:
        driver.get(link)
        time.sleep(0.5)
        address = driver.find_element(By.CLASS_NAME,'address').text
        addresses.append(address)
        (lat,long)=get_location(address)
        lats.append(lat)
        longs.append(long)

    print(f'Page {page} done')

# Print the number of links extracted
print(f'{len(links)} links extracted')

# Close the web driver
driver.quit()

Page 1 done
Page 2 done
Page 3 done
Page 4 done
Page 5 done
Page 6 done
Page 7 done
Page 8 done
Page 9 done
Page 10 done
Page 11 done
Page 12 done
Page 13 done
Page 14 done
Page 15 done
Page 16 done
Page 17 done
Page 18 done
Page 19 done
Page 20 done
Page 21 done
Page 22 done
Page 23 done
Page 24 done
Page 25 done
Page 26 done
Page 27 done
Page 28 done
Page 29 done
Page 30 done
Page 31 done
Page 32 done
Page 33 done
Page 34 done
Page 35 done
Page 36 done
Page 37 done
Page 38 done
Page 39 done
Page 40 done
Page 41 done
Page 42 done
Page 43 done
Page 44 done
Page 45 done
Page 46 done
Page 47 done
Page 48 done
Page 49 done
Page 50 done
Page 51 done
Page 52 done
Page 53 done
Page 54 done
Page 55 done
Page 56 done
Page 57 done
Page 58 done
Page 59 done
Page 60 done
Page 61 done
Page 62 done
Page 63 done
Page 64 done
Page 65 done
Page 66 done
Page 67 done
Page 68 done
Page 69 done
Page 70 done
1050 links extracted


In [16]:
import pandas as pd
#tạo dataframe và lưu vào file csv
df = pd.DataFrame({'link': links, 'price': prices,'area':areas,'room':rooms,'toilet':toilets,'address':addresses,'lat':lats,'long':longs})
print(df.head())
df.to_csv('raw_SMALL.csv', index=False)

                                                link           price   area  \
0  https://mogi.vn/quan-ngu-hanh-son/mua-can-ho-c...  3 tỷ 449 triệu  82 m2   
1  https://mogi.vn/quan-ngu-hanh-son/mua-can-ho-c...  3 tỷ 499 triệu  82 m2   
2  https://mogi.vn/quan-ngu-hanh-son/mua-can-ho-c...  3 tỷ 500 triệu  82 m2   
3  https://mogi.vn/quan-ngu-hanh-son/mua-can-ho-c...  3 tỷ 500 triệu  83 m2   
4  https://mogi.vn/quan-ngu-hanh-son/mua-can-ho-c...  3 tỷ 500 triệu  82 m2   

   room toilet                                            address       lat  \
0  2 PN   2 WC  Phạm Kiệt, Phường Khuê Mỹ, Quận Ngũ Hành Sơn, ...  16.03851   
1  2 PN   2 WC  Phạm Kiệt, Phường Khuê Mỹ, Quận Ngũ Hành Sơn, ...  16.03851   
2  2 PN   2 WC  Phạm Kiệt, Phường Khuê Mỹ, Quận Ngũ Hành Sơn, ...  16.03851   
3  2 PN   2 WC  Phạm Kiệt, Phường Khuê Mỹ, Quận Ngũ Hành Sơn, ...  16.03851   
4  2 PN   2 WC  Phạm Kiệt, Phường Khuê Mỹ, Quận Ngũ Hành Sơn, ...  16.03851   

        long  
0  108.24755  
1  108.24755  
2  10